# Manipulation des images

In [ ]:
import numpy as np
import scipy as scp
import holoviews as hv
import panel as pn
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

## Images Couleur 

Numpy permet de charger simplement des images couleur. Holoview permet de les afficher.

In [ ]:
Mi=chargeData('Minotaure').astype('uint8')
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
hv.RGB(Mi).opts(**options1)

In [ ]:
im=chargeData('Cartoon').astype('uint8')
hv.RGB(im).opts(**options1)

La variable Im est un tableau à 3 dimensions.

In [ ]:
n1,n2,n3=np.shape(im)
print(n1,n2,n3)

On peut afficher les différents canaux de couleurs avec la commande hv.Raster faite pour les images monochromes.

In [ ]:
im1=im[:,:,0]
im2=im[:,:,1]
im3=im[:,:,2]
hv.Raster(im1).opts(cmap='kr',width=300,height=300,xaxis=None,yaxis=None)+\
hv.Raster(im2).opts(cmap='kg',width=300,height=300,xaxis=None,yaxis=None)+\
hv.Raster(im3).opts(cmap='kb',width=300,height=300,xaxis=None,yaxis=None)

On peut aussi afficher une image avec la commande hv.Image, la différence entre les deux commandes est essentiellement le choix de l'échelle des variables et du sens des axes.

In [ ]:
pn.Row(hv.Image(im1).opts(width=400,height=400,cmap='gray',toolbar=None)\
       ,hv.Raster(im1).opts(width=400,height=400,cmap='gray',toolbar=None))

## Ajouter du bruit à une image

On peut ajouter un bruit gaussien à une image couleur :

In [ ]:
n1,n2,n3=np.shape(im)
bruitcouleur=np.random.randn(n1,n2,3)
sigma=50
imb=im+sigma*bruitcouleur

Avant d'afficher le résultat, il faut avoir conscience qu'une image est normalement un tableau de valeurs entières comprises entre 0 et 255. Quand on modifie le tableau, par exemple en lui ajoutant un bruit, les valeurs peuvent ne pas être entières et ne plus être entre 0 et 255 

In [ ]:
print(np.min(imb),np.max(imb))

Les commandes hv.Raster et hv.Image permettent quand même un affichage :

In [ ]:
pn.Row(hv.Image(imb[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None)\
       ,hv.Raster(imb[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None))

En revanche, la commande hv.RGB ne fonctionne pas tout à fait comme on le voudrait.

In [ ]:
hv.RGB(imb)

Le problème d'un tel affichage est que par défaut, holoview associe à blanc la plus forte valeur et à noir la plus faible valeur. Or ces valeurs peuvent varier, d'une réalisation du bruit à l'autre ou être différente de celles de l'image originale. Cette différence peut créer une différence de contraste artificielle d'une image à l'autre :  

In [ ]:
pn.Row(hv.Image(imb[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None),\
hv.Image(im[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None))

On peut limiter ces effets en clipant l'image entre 0 et 255 :

In [ ]:
imb=np.clip(imb,0,255)
pn.Row(hv.Image(imb[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None),\
hv.Image(im[:,:,0]).opts(width=400,height=400,cmap='gray',toolbar=None))

Vous pouyez vérifier que ce cliping ne suffit pas à assurer un bon affichage en couleur via la fonction hv.RBG. Il faut en plus convertir les valeurs du tableau en entiers. 

Une manière de le faire est d'utiliser la commande astype.

In [ ]:
imb=imb.astype('uint8')
hv.RGB(imb).opts(**options1)
im_cartoon_bruitee = Image.fromarray(imb)
im_cartoon_bruitee.save('MinotaureBruite.jpeg')

## Sauver un tableau sous forme d'un fichier image

Pour sauver un tableau sous forme d'image, il faut d'abord s'assurer que le tableau ne comprend que des entiers de 0 à 255 et que le type des éléments du tableau est correct. On peut utiliser pour cela les fonctions np.clip et astype vues plus haut. Ensuite on procède comme suit : 

In [ ]:
im_cartoon_bruitee = Image.fromarray(imb)
im_cartoon_bruitee.save('Cartoon_bruitee.jpeg')

## Image couleur vers Image monochrome de luminance

La luminance d'une image se calcule par combinaison linéaire des trois canaux de couleur. Les coefficients de la combinaison linéaire sont standardisés.

In [ ]:
n1,n2,n3=np.shape(im)
iml=0.2126*im[:,:,0]+0.7152*im[:,:,1]+0.0722*im[:,:,2]
iml=np.reshape(iml,(n1,n2))
print(np.shape(iml))

In [ ]:
hv.Image(iml).opts(cmap='gray',width=400,height=400,xaxis=None,yaxis=None,toolbar=None)

## Zoom interactif

On va maintenant voir comment on affiche un zoom interactif sur plusieurs images simultanément.

Nous allons ici travailler par simplicité sur une image monochrome, l'image de luminance créée plus haut. 

La commande suivante permet de créer une image source, mais elle n'affiche rien.

In [ ]:
source = hv.Raster(iml).opts(cmap='gray',xaxis=None,yaxis=None)

Si on veut afficher la source il suffit de taper :

In [ ]:
source

Les lignes suivantes permettent de créer une sous image centrée d'une image im au point (x,y) de taille définie par la variable Taille, ici 60. La fonction renvoie une image 

In [ ]:
def crop(x,y,im):
    Taille=50
    xx=int(x)
    yy=int(y)
    y0,y1,x0,x1 = max(yy-Taille,0),min(yy+Taille,im.shape[0]),max(xx-Taille,0),min(xx+Taille,im.shape[1])
    imcrop = im[y0:y1,x0:x1]
    options = dict(cmap='gray',xaxis=None,yaxis=None)
    return hv.Raster(imcrop,kdims=['x_zoom','y_zoom']).opts(**options)

In [ ]:
crop(230,350,iml)

La commande suivante construit un pointeur qui renvoie de manière interactive, l'abscisse et l'ordonnée du curseur quand il est sur l'image source. Les valeurs par défaut sont fixée ici à x=20 à y=20.

In [ ]:
pointer = hv.streams.PointerXY(source=source,x=20,y=20)

La ligne suivante permet de créer un zoom interactif, construit sur le pointeur 

In [ ]:
viewcrop=hv.DynamicMap(lambda x,y: crop(x,y,iml), streams=[pointer])

On peut ensuite afficher l'image source et le zoom intéractif l'un à côté de l'autre.

In [ ]:
(source+viewcrop).opts(toolbar=None)

On peut afficher des zooms de plusiurs images en utilisant le même pointeur. On crée une image bruitée...

In [ ]:
imlb=iml+20*np.random.randn(n1,n2)
options2 = dict(cmap='gray',xaxis=None,yaxis=None)
hv.Raster(imlb).opts(**options2)

... et on crée le zoom interactif. On peut ainsi créer et mettre en regard autant de zooms que l'on veut, tous définis par le même pointeur.

In [ ]:
viewcrop2=hv.DynamicMap(lambda x,y: crop(x,y,imlb), streams=[pointer])
(source+viewcrop+viewcrop2).opts(toolbar=None)